In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
# Copied from Part 1.
def lbp_pixel(img, r, c):
    h, w = img.shape
    offsets = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]
    pixel = img[r, c]
    res = 0
    for (dr, dc) in offsets:
        nr, nc = r + dr, c + dc
        if nr < 0 or nr > h - 1 or nc < 0 or nc > w - 1:
            neighbor = 0
        else:
            neighbor = img[nr, nc]
        code = 0 if neighbor <= pixel else 1
        res = (res << 1) + code
    return res
def calc_lbp_img(img):
    imgLBP = np.zeros_like(img)
    h, w = img.shape
    for r in range(h):
        for c in range(w):
            imgLBP[r, c] = lbp_pixel(img, r, c)
    return imgLBP
def calc_lbp_hist(img, normalize=True, mask=None):
    imgLBP = calc_lbp_img(img)
    hist = cv2.calcHist([imgLBP], [0], mask, [256], [0, 256])
    if normalize:
        hist /= hist.sum()
    return hist.squeeze()

In [3]:
TrainDir = './data/train'
TestDir = './data/test'
Categories = ['asphaltroad', 'brickwall', 'gravelroad', 'woodentable']

In [4]:
# Read training set.
TrainImgs = []
list_labels = []
for category in Categories:
    for i in range(3):
        path = os.path.join(TrainDir, '{}_{}.jpg'.format(category, i + 1))
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        TrainImgs.append(img)
        list_labels.append(category)

In [5]:
# Compute feature vectors of training set.
list_lbp_hist = []
for img in TrainImgs:
    hist = calc_lbp_hist(img, normalize=True)
    list_lbp_hist.append(hist)

In [6]:
# Read test set.
TestImgs = []
TestLabels = []
for category in Categories:
    path = os.path.join(TestDir, '{}_4.jpg'.format(category))
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
    img = np.array(img)
    TestImgs.append(img)
    TestLabels.append(category)

In [7]:
# Compute feature vectors of test set.
TestHists = []
for img in TestImgs:
    hist = calc_lbp_hist(img, normalize=True)
    TestHists.append(hist)

In [8]:
# Copied from Part 1.
def lbp_histograms_distance(h1, h2):
    diff = 0.0
    for i in range(256):
        diff += abs(h1[i] - h2[i])
    return diff

In [9]:
np.set_printoptions(precision=3)
for i in range(4): # 4 test images
    distances = []
    histTest = TestHists[i]
    for j in range(4 * 3): # 12 training images
        histTrain = list_lbp_hist[j]
        distances.append(lbp_histograms_distance(histTest, histTrain))
    pred = np.argmin(np.array(distances))
    print('Ground-Truth: {}, Prediction: {}'
          .format(TestLabels[i], list_labels[pred]))
    print('Distances: ', np.array(distances), '\n')

Ground-Truth: asphaltroad, Prediction: asphaltroad
Distances:  [0.09  0.213 0.274 0.471 0.403 0.477 0.323 0.353 0.359 0.468 0.482 0.495] 

Ground-Truth: brickwall, Prediction: brickwall
Distances:  [0.522 0.491 0.501 0.078 0.224 0.082 0.425 0.615 0.499 0.475 0.398 0.423] 

Ground-Truth: gravelroad, Prediction: gravelroad
Distances:  [0.334 0.275 0.33  0.443 0.342 0.436 0.166 0.205 0.161 0.51  0.541 0.55 ] 

Ground-Truth: woodentable, Prediction: woodentable
Distances:  [0.453 0.382 0.339 0.366 0.322 0.341 0.47  0.557 0.558 0.187 0.058 0.081] 

